## Exercise 4: Data Warehouse Querying and Basic Geospatial Operations

Skills: 
* Query data warehouse table
* Use dictionary to map values

References: 
* https://docs.calitp.org/data-infra/analytics_new_analysts/05-spatial-analysis-basics.html
* https://docs.calitp.org/data-infra/analytics_new_analysts/06-spatial-analysis-intro.html
* https://docs.calitp.org/data-infra/analytics_new_analysts/07-spatial-analysis-intermediate.html
* https://github.com/jorisvandenbossche/geopandas-tutorial

In [1]:
import geopandas as gpd
import pandas as pd
import os

#os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)
pd.set_option("display.max_rows", 20)

#from calitp_data_analysis.tables import tbls
#from calitp_data_analysis.sql import query_sql
from siuba import *
FOLDER = "./data/"
FILE_NAME = "exercise_4_stops_sample.parquet"
stops=pd.read_parquet(f"{FOLDER}{FILE_NAME}")

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_117/433137083.py:1: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling Py

## Query a table, turn it into a gdf

You will query the warehouse table for 2 operators, Caltrain and Merced. A `feed_key` is a hash identifier, there's no real meaning to it, but it uniquely identifies a feed for that day.

The `feed_key` values for those 2 operators for 6/1/2022 are provided. 

* Query `mart_gtfs.dim_stops`
* Filter to the feed keys of interest
* Select these columns: `feed_key`, `stop_id`, `stop_lat`, `stop_lon`, `stop_name`
* Return as a dataframe using `collect()`
* Turn the point data into geometry with `geopandas`: [docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.points_from_xy.html)

In [ ]:
FEEDS = [
    "25c6505166c01099b2f6f2de173e20b9", # Caltrain
    "52639f09eb535f75b33d2c6a654cb89e", # Merced
]

stops = pd.read_parquet('./data/exercise_4_stops_sample.parquet')
stops = (
    tbls.mart_gtfs.dim_stops()
    >> filter(_.feed_key.isin(FEEDS))
    >> select(_.feed_key, _.stop_id, 
             _.stop_lat, _.stop_lon, _.stop_name)
    >> arrange(_.feed_key, _.stop_id, 
               _.stop_lat, _.stop_lon)
    >> collect() 
)

In [2]:
stops.head()

,feed_key,stop_id,stop_lat,stop_lon,stop_name
0,52639f09eb535f75b33d2c6a654cb89e,768319,37.067249,-120.855585,G St @ Davita Dialysis (southbound)
1,52639f09eb535f75b33d2c6a654cb89e,768398,37.345519,-120.608472,Atwater Transpo
2,25c6505166c01099b2f6f2de173e20b9,san_carlos,37.508033,-122.260200,San Carlos
3,52639f09eb535f75b33d2c6a654cb89e,768199,37.382422,-120.723304,Main St @ F St (northbound)
4,52639f09eb535f75b33d2c6a654cb89e,768413,37.353220,-120.613861,Winton @ Grove (southbound)


In [4]:
#Turn the point data into geometry with geopandas???????

gdf = gpd.GeoDataFrame(
    stops, 
    geometry=gpd.points_from_xy(stops['stop_lon'], stops['stop_lat']),
    crs='EPSG:4326'
)

In [5]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Use a dictionary to map values

* Create a new column called `operator` where `feed_key` is associated with its operator name.
* First, write a function to do it.
* Then, use a dictionary to do it (create new column called `agency`).
* Double check that `operator` and `agency` show the same values. Use `assert` to check.
    * `assert df.operator == df.agency`
* Hint: https://docs.calitp.org/data-infra/analytics_new_analysts/02-data-analysis-intermediate.html

In [6]:
stops.feed_key.value_counts()

52639f09eb535f75b33d2c6a654cb89e    197
25c6505166c01099b2f6f2de173e20b9     35
Name: feed_key, dtype: int64

In [7]:
def name1(row):
    if row.feed_key == '52639f09eb535f75b33d2c6a654cb89e':
        return 'a'
    elif row.feed_key == '25c6505166c01099b2f6f2de173e20b9':
        return 'b'
    else:
        return 'other'
stops['operator'] = stops.apply(name1, axis = 1)

In [8]:
#using a dictionary
name2={'52639f09eb535f75b33d2c6a654cb89e': 'a', '25c6505166c01099b2f6f2de173e20b9': 'b'}

stops['agency'] = stops.feed_key.map(name2)

In [9]:
#two methods got the same answer. do i have to type 'assert' in the front???
stops.operator == stops.agency

0      True
1      True
2      True
3      True
4      True
       ... 
227    True
228    True
229    True
230    True
231    True
Length: 232, dtype: bool

In [10]:
#another way to do this
assert(stops.operator == stops.agency).all()

## Turn lat/lon into point geometry

* There is a [function in shared_utils](https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/geography_utils.py#L170-L192) that does it. Show the steps within the function (the long way), and also create the `geometry` column using `shared_utils`.
* Use `geography_utils.create_point_geometry??` to see what goes into that function, and what that function looks like under the hood.

In [13]:
# Rename columns
stops.rename(columns = {'stop_lon': 'longitude', 'stop_lat':'latitude'}, inplace=True)

# Create geometry column
gdf = gpd.points_from_xy(stops.longitude, stops.latitude, crs="EPSG:4326")
stops['geometry']=gdf


In [14]:
stops.head()

,feed_key,stop_id,latitude,longitude,stop_name,operator,agency,geometry
0,52639f09eb535f75b33d2c6a654cb89e,768319,37.067249,-120.855585,G St @ Davita Dialysis (southbound),a,a,POINT (-120.85559 37.06725)
1,52639f09eb535f75b33d2c6a654cb89e,768398,37.345519,-120.608472,Atwater Transpo,a,a,POINT (-120.60847 37.34552)
2,25c6505166c01099b2f6f2de173e20b9,san_carlos,37.508033,-122.260200,San Carlos,b,b,POINT (-122.26020 37.50803)
3,52639f09eb535f75b33d2c6a654cb89e,768199,37.382422,-120.723304,Main St @ F St (northbound),a,a,POINT (-120.72330 37.38242)
4,52639f09eb535f75b33d2c6a654cb89e,768413,37.353220,-120.613861,Winton @ Grove (southbound),a,a,POINT (-120.61386 37.35322)


In [ ]:
#Another way
# Default CRS for stop_lon, stop_lat is WGS84
stops = stops.assign(geometry=gpd.points_from_xy(df[stop_lon], df[stop_lat], crs="EPSG:4326")

In [ ]:
# Another way: Use geography_utils.create_point_geometry?? to see what goes into that function,
#and what that function looks like under the hood.
def create_point_geometry(
    stops: pd.DataFrame,
    longitude_col: str = "stop_lon",
    latitude_col: str = "stop_lat",
    crs: str = crs="EPSG:4326",
) -> gpd.GeoDataFrame

Basic stuff about a geodataframe.

A gdf would have a coordinate reference system that converts the points or lines into a place on the spherical Earth. The most common CRS is called `WGS 84`, and its code is `EPSG:4326`. This is what you'd see when you use Google Maps to find lat/lon of a place.

[Read](https://desktop.arcgis.com/en/arcmap/latest/map/projections/about-geographic-coordinate-systems.htm) about the `WGS 84` geographic coordinate system.

[Read](https://desktop.arcgis.com/en/arcmap/latest/map/projections/about-projected-coordinate-systems.htm) about projected coordinate reference systems, which is essentially about flattening our spherical Earth into a 2D plane so we can measure distances and whatnot.

* Is it a pandas dataframe or a geopandas geodataframe?: `type(gdf)`
* Coordinate reference system: `gdf.crs`
* gdfs must have a geometry column. Find the name of the column that is geometry: `gdf.geometry.name`
* Project the coordinate reference system to something else: `gdf = gdf.to_crs("EPSG:2229")` and check.

In [15]:
type(gdf)

geopandas.array.GeometryArray

In [16]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [18]:
stops.geometry.name

'geometry'

In [ ]:
gdf = gdf.to_crs("EPSG:2229")

* This GitHub repo has several `geopandas` tutorials that covers basic spatial concepts: https://github.com/jorisvandenbossche/geopandas-tutorial. 
* Skim through the notebooks to see some of the concepts demonstrated, although to actually run the notebooks, you can click on `launch binder` in the repo's README to do so.

## Spatial Join (which points fall into which polygon)

This URL gives you CA county boundaries: https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-county-boundaries/explore?location=37.246136%2C-119.002032%2C6.12

* Go to "I want to use this" > View API Resources > copy link for geojson
* Read in the geojson with `geopandas` and make it a geodataframe: `gpd.read_file(LONG_URL_PATH)`
* Double check that the coordinate reference system is the same for both gdfs using `gdf.crs`. If not, change it so they are the same.
* Spatial join stops to counties: [docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html)
    * Play with inner join or left join, what's the difference? Which one do you want?
    * Play with switching around the left_df and right_df, what's the right order?
* By county: count number of stops and stops per sq_mi.
    * Hint 1: Start with a CRS with units in feet or meters, then do a conversion to sq mi. [CRS in shared_utils](https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/geography_utils.py)
    * Hint 2: to find area, you can create a new column and calculate `gdf.geometry.area`. [geometry manipulations docs](https://geopandas.org/en/stable/docs/user_guide/geometric_manipulations.html)

In [19]:
counties = gpd.read_file('https://services1.arcgis.com/jUJYIo9tSA7EHvfZ/arcgis/rest/services/California_County_Boundaries/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')

In [20]:
counties.head()

,OBJECTID,COUNTY_NAME,COUNTY_ABBREV,COUNTY_NUM,COUNTY_CODE,COUNTY_FIPS,ISLAND,Shape__Area,Shape__Length,GlobalID,geometry
0,1,Alameda,ALA,1,01,001,NaN,3.402787e+09,308998.650766,e6f92268-d2dd-4cfb-8b79-5b4b2f07c559,"POLYGON ((-122.27125 37.90503, -122.27024 37.9..."
1,2,Alpine,ALP,2,02,003,NaN,3.146939e+09,274888.492411,870479b2-480a-494b-8352-ad60578839c1,"POLYGON ((-119.58667 38.71420, -119.58653 38.7..."
2,3,Amador,AMA,3,03,005,NaN,2.562635e+09,361708.438013,4f45b3a6-be10-461c-8945-6b2aaa7119f6,"POLYGON ((-120.07246 38.70276, -120.07249 38.6..."
3,4,Butte,BUT,4,04,007,NaN,7.339348e+09,526547.115238,44fba680-aecc-4e04-a499-29d69affbd4a,"POLYGON ((-121.07661 39.59729, -121.07945 39.5..."
4,5,Calaveras,CAL,5,05,009,NaN,4.351069e+09,370637.578323,d11ef739-4a1e-414e-bfd1-e7dcd56cd61e,"POLYGON ((-120.01792 38.43586, -120.01788 38.4..."


In [ ]:
# If the CRS is not set after checking it with gdf.crs

#gdf = gdf.set_crs('EPSG:4326')


In [22]:
counties = counties.to_crs('EPSG:4326')

In [23]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [24]:
merge1 = pd.merge(counties, stops, on = 'geometry',
    how = 'inner', validate = 'm:1')

In [26]:
merge2 = pd.merge(counties, stops, left_on = 'COUNTY_NAME',
    right_on = 'geometry', how = 'left', validate = 'm:1')

In [30]:
gpd.sjoin(stops, counties, how='inner', predicate='intersects', lsuffix='left', rsuffix='right')

ValueError: 'left_df' should be GeoDataFrame, got <class 'pandas.core.frame.DataFrame'>